## Imports and setup

In [ ]:
# Create or update conda environment
# !conda env create -f ../environment.yaml
!conda env update -f ../environment.yaml
# !conda remove --name amadeus-ex-machina --all

In [ ]:
# Activate conda environment
!conda init
!conda activate amadeus-ex-machina

In [ ]:
# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Class/model imports
from data.data_loader import MirDataProcessor
from utils.model_utils import get_device
from solver import Solver
import data.youtube_download as youtube_download

# Package imports
import torch

# Select device
device = get_device()
print(f"Device is {device}")

## Download and process data

In [ ]:
# Download and build useable train/test data out of the MIR Billboard dataset
data_processer = MirDataProcessor(download=True, batch_size=64)
data_processer.process_data()

# Create data loeaders for train and test set
train_loader, test_loader, num_classes = data_processer.build_data_loaders(device=device)

print(f"Number of classes: {num_classes}")

Preparing model data...
Loading the combined CSV file...
Separating features and labels...
Scaling features using MinMaxScaler...
Encoding labels using LabelEncoder...
Saving the scaler to /Users/dananabulsi/Desktop/CODING/GitHub-Repos/amadeus-ex-machina/data/processed/scaler.pkl...
Saving the label encoder to /Users/dananabulsi/Desktop/CODING/GitHub-Repos/amadeus-ex-machina/data/processed/label_encoder.pkl...
Splitting data into training and testing sets...
Data preparation complete.
Number of classes determined: 813
Creating TensorDatasets for training and testing data...
Creating DataLoaders for training and testing datasets...
Data loaders are ready for training and testing.
Number of classes: 813


In [ ]:
# Set billboard data path
billboard_data_path = "../data/raw/McGill-Billboard"

# Process lab files in the extracted directory
youtube_download.process_lab_files(billboard_data_path)

## Build and train models

In [8]:
# Initialize Solver for MLPChordClassifier
mlp_chord_classifier_solver = Solver(
    model_type="MLPChordClassifier",
    model_kwargs={"input_size": 24, "num_classes": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

# Train and evaluate the model
mlp_chord_classifier_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# Initialize Solver for CRNNModel
crnn_model_solver = Solver(
    model_type="CRNNModel",
    model_kwargs={"input_features": 24, "num_classes": num_classes, "hidden_size": 128},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

crnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

In [ ]:
# Initialize Solver for CNNModel
cnn_model_solver = Solver(
    model_type="CNNModel",
    model_kwargs={"input_channels": 24, "num_classes": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

cnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

In [ ]:
# Initialize Solver for RNNModel
rnn_model_solver = Solver(
    model_type="RNNModel",
    model_kwargs={"input_size": 24, "hidden_size": 128, "output_size": num_classes},
    device=device,
    batch_size=128,
    learning_rate=0.001,
    epochs=20,
)

rnn_model_solver.train_and_evaluate(train_loader, test_loader, plot_results=True)

## Run inference

In [ ]:
scaler = data_processor.scaler
label_encoder = data_processor.label_encoder
chroma_path = "/my/path/to/amadeus-ex-machina/data/raw/McGill-Billboard/0003/bothchroma.csv"

# Run inference using the trained model
solver.run_inference(
    chroma_path,
    scaler,
    label_encoder,
)